<a href="https://colab.research.google.com/github/willyandan/truco.ai/blob/master/Truco_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import random

In [0]:
class Carta:
    valor, naipe = "", ""
    forca = 0
    def __init__(self, valor, forca, naipe):
        self.valor = valor
        self.naipe = naipe
        self.forca = forca
    def __str__(self):
        return self.valor+"["+self.naipe+"]"

In [0]:
class Player:
    #SER GAMER SER UM JOGADOR...
    mao = []
    pontos=0
    def __init__(self):
        mao = []
        self.pontos = 0
    def add_ponto(self):
        self.pontos += 1
    def set_mao(self, mao):
        self.mao = mao

In [0]:
class Jogo:
    naipes = ['♥', '♣', '♠', '♦']
    baralho = []
    jogadores = []
    manilha = None
    analise = []
    def __init__(self):
        self.jogadores = []
        self.baralho = []
        self.jogadores.append(Player())
        self.jogadores.append(Player())
    def preenche_baralho(self):
        self.baralho = []
        for n in self.naipes:
            i=1
            for v in "4567QJKA23":
                self.baralho.append(Carta(v, i, n))
                i+=1
    def partida(self):
        self.analise = []
        self.preenche_baralho()
        random.shuffle(self.baralho)
        self.manilha = self.escolhe_carta()
        for player in self.jogadores:
            player.set_mao(self.dar_mao())
        return self.manilha
    
    def rodada(self, player, id_carta):
        carta = player.mao[id_carta]
        player.mao.pop(id_carta)
        self.analise.append({"p":player, "carta":carta})
    
    def pontua(self):
        if len(self.analise) == 0: return
        
        rodada_1 = self.analise[0:2]
        rodada_2 = self.analise[2:4]
        rodada_3 = self.analise[4:6]
        campeoes = {1:None, 2: None, 3:None}
        for x, rodada in enumerate([rodada_1, rodada_2, rodada_3], start=1):
            campeoes[x] = self.compara_carta(rodada)
            
        if not campeoes[1] and not campeoes[2] and not campeoes[3]:
            pass
        elif not campeoes[1] and not campeoes[2]:
            campeoes[3].add_ponto()
        elif not campeoes[1]:
            campeoes[2].add_ponto()
        elif not campeoes[2]:
            campeoes[1].add_ponto()
        elif not campeoes[3]:
            campeoes[1].add_ponto()
        else:
            if campeoes[1] == campeoes[2] or campeoes[1] == campeoes[3]:
                campeoes[1].add_ponto()
            else:
                campeoes[2].add_ponto()
        self.analise = []
    def escolhe_carta(self):
        card = self.baralho[-1]
        self.baralho.pop()
        return card

    def dar_mao(self):
        c1 = self.escolhe_carta()
        c2 = self.escolhe_carta()
        c3 = self.escolhe_carta()
        return [c1, c2, c3]
    
    def compara_carta(self, rodada):
        c1 = rodada[0]["carta"]
        c2 = rodada[1]["carta"]
        p1 = rodada[0]["p"]
        p2 = rodada[1]["p"]
        fortona = "4567QJKA23"[self.manilha.forca % 10]
        naipe_forca = {'♥':2, '♣':3, '♠':1, '♦':0}
        
        if c1.valor == fortona and c2.valor == fortona:
            #empate de manilha....
            return p1 if naipe_forca[c1.naipe] > naipe_forca[c2.naipe] else p2
        elif c1.valor == fortona:
            #c1 ganha
            return p1
        elif c2.valor == fortona:
            #c2 ganha
            return p2
        elif c1.forca == c2.forca:
            return None
        else:
            #jogo normal
            return p1 if c1.forca > c2.forca else p2
            

In [0]:
def make_jogo():
  pontos = {}
  jogo = Jogo()
  for j in jogo.jogadores:
    pontos[j] = []
  manilha = jogo.partida()
  for i in range(3):
      for p in jogo.jogadores:
        idx_carta = random.randint(0, len(p.mao) -1 )
        carta_jogada = p.mao[idx_carta]
        jogo.rodada(p, idx_carta)
        pontos[p].append(np.array([ 
                          [manilha.forca],
                          [carta_jogada.forca],
                          [ {'♥':2, '♣':3, '♠':1, '♦':0}[carta_jogada.naipe] ], 
                          [len(p.mao) + 1],
                          [idx_carta]
                        ]))
                
  jogo.pontua()
  if jogo.jogadores[0].pontos > jogo.jogadores[1].pontos:
    return np.array(pontos[jogo.jogadores[0]])
  elif jogo.jogadores[1].pontos > jogo.jogadores[0].pontos:
    return np.array(pontos[jogo.jogadores[1]])

In [14]:
'''
FEATURES
1- Manilha
2- Cartas da mão (ou cartas disponiveis)
3- Se o adversário já jogou a sua carta
4- Carta do adversário (0 se ele não jogou ainda)
5- Se ele venceu a partida ou não
'''

array([[22.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.utils import normalize

Using TensorFlow backend.
